General setup:

In [1]:
library("extRemes")
library(ncdf4)                  # needed to read netcdf data
library(ncdf4.helpers)          # additional support functions for netcdf data
library(fields)                 # provides image.plot()
library(rgdal)                  # load shapefiles

setwd("~/UKCP18/ukcp-other/extremal-dependence-data/")

Loading required package: Lmoments

Loading required package: distillery


Attaching package: ‘extRemes’


The following objects are masked from ‘package:stats’:

    qqnorm, qqplot


Loading required package: spam

Loading required package: dotCall64

Loading required package: grid

Spam version 2.6-0 (2020-12-14) is loaded.
Type 'help( Spam)' or 'demo( spam)' for a short introduction 
and overview of this package.
Help for individual functions is also obtained by adding the
suffix '.spam' to the function name, e.g. 'help( chol.spam)'.


Attaching package: ‘spam’


The following objects are masked from ‘package:base’:

    backsolve, forwardsolve


Loading required package: viridis

Loading required package: viridisLite

See https://github.com/NCAR/Fields for
 an extensive vignette, other supplements and source code 

Loading required package: sp

Warning message:
“package ‘sp’ was built under R version 4.0.3”
rgdal: version: 1.5-16, (SVN revision 1050)
Geospatial Data Abstraction Lib

In [2]:
nc2 <- nc_open("land-sea_mask_uk_12km-rll.nc")
land_mask <- ncvar_get(nc2)
land_mask_ind <- which(!is.na(land_mask), arr.ind=T)
#View(land_mask_ind)

In [3]:
CHI_CUTOFF = 0.95
CHI_TYPE = "chi"

Helper-functions:

In [4]:
get_loc_lat_ids_from_coords <- function(lon, lat){
    c(which(dimnames(tasmax)$rlon == lon), which(dimnames(tasmax)$rlat == lat))
}

get_nearest_date_idx <- function(date){
     
    date = as.Date(date, format = "%Y-%m-%d")
    
    if(format(date, "%d") == "31"){
        date = as.Date(paste0(format(date, "%Y-%m"), "-30"))
    }
    return(
        which(dimnames(tasmax)$date == format(date, "%Y-%m-%d"))
    )
}

get_values_over_time_by_loc_ids <- function(lon_id, lat_id, start_date, end_date){
    upper_date_bound_idx <- get_nearest_date_idx(start_date) # round right?
    lower_date_bound_idx <- get_nearest_date_idx(end_date)
    return(
        tasmax[lon_id, lat_id, upper_date_bound_idx:lower_date_bound_idx]
    )
}

get_chi_two_series <- function(x, y){
    return(taildep(x, y, CHI_CUTOFF, type = CHI_TYPE))
}

Date specific stuff:

In [5]:
filenames = c(
    "tasmax_rcp85_land-rcm_uk_12km-rll_01_day_20201201-20301130.nc",
    "tasmax_rcp85_land-rcm_uk_12km-rll_01_day_20301201-20401130.nc",
    "tasmax_rcp85_land-rcm_uk_12km-rll_01_day_20401201-20501130.nc"
)

start_dates = c(
    "2020-12-01",
    "2030-12-01",
    "2040-12-01"
)

end_dates = c(
    "2030-11-30", 
    "2040-11-30",
    "2050-11-30"
)




Main iteration:

In [6]:
for(range_index in 1:3){
    
    fnm <- filenames[range_index]
    start_date <- start_dates[range_index]
    end_date <- end_dates[range_index]
    
    # File reading stuff
    nc <- nc_open(fnm)
    tasmax <- ncvar_get(nc, "tasmax")
    dimnames(tasmax) <- list("rlon" = ncvar_get(nc, "grid_longitude") - 360,
                          "rlat" = ncvar_get(nc, "grid_latitude"),
                          "date" = substr(nc.get.time.series(nc, "tasmax"),1,10))
    nc_close(nc); remove(nc)
    
    # Main calculation-iterations
    
    all_chi_vals <- matrix(NA, dim(land_mask_ind)[1], dim(land_mask_ind)[1])

    for(i in 1:dim(land_mask_ind)[1]){

        base_lon_id = land_mask_ind[i, 1]
        base_lat_id = land_mask_ind[i, 2]

        base_loc_vals <- get_values_over_time_by_loc_ids(base_lon_id, base_lat_id, start_date, end_date)

        chi_vals <- matrix(NA, dim(tasmax)[1], dim(tasmax)[2])

        for(j in 1:dim(land_mask_ind)[1]){

            comp_lon_id = land_mask_ind[j, 1]
            comp_lat_id = land_mask_ind[j, 2]

            comp_loc_vals <- get_values_over_time_by_loc_ids(comp_lon_id, comp_lat_id, start_date, end_date)

            all_chi_vals[i,j] <- get_chi_two_series(base_loc_vals, comp_loc_vals)
            #chi_vals[comp_lon_id, comp_lat_id] <- get_chi_two_series(base_loc_vals, comp_loc_vals)
        }
        #assign(paste("chi_vals", base_lat_id, base_lon_id, sep = "_"), chi_vals)
    }

    write.table(all_chi_vals, file = paste0("~/chi_vals_", start_date, "_", end_date, ".txt"), row.names=FALSE, col.names=FALSE)
    write.table(land_mask_ind, file = paste0("~/land_mask_ind_", start_date, "_", end_date, ".txt"))
    write(dimnames(tasmax)$rlon, file = paste0("~/lon_coords_", start_date, "_", end_date, ".txt"))
    write(dimnames(tasmax)$rlat, file = paste0("~/lat_coords_", start_date, "_", end_date, ".txt"))
    
}

